In [5]:
import torch
import random

import torch.nn.functional as F



In [6]:
BATCHSIZE = 10
TARGET = [0.34, 0.2]


#r = torch.tensor([[random.uniform(0, 1), random.uniform(0, 1)]], dtype=torch.double)  
def dif(tensor, target):
  target = torch.tensor([target], dtype=torch.double)
  r = target - tensor 
  r = torch.abs(r)
  r = torch.sum(r)
  #r = torch.clamp(r,0,1)
  return torch.tensor([[r]], dtype=torch.double)
  


def getSample():
  return torch.tensor([[random.uniform(0, 1), random.uniform(0, 1)]], dtype=torch.double)  

X = torch.tensor([], dtype=torch.double)
Y = torch.tensor([], dtype=torch.double)
# Create a Standardize transform using the data

for i in range(BATCHSIZE):
  s = getSample()
  X = torch.cat((X, s), dim=0)

for i in X:
  s = dif(i, TARGET)
  Y = torch.cat((Y, s), dim=0)
  
Y = F.normalize(Y, dim=None, p=2)

print(Y)


list(zip(X, Y))


tensor([[0.2994],
        [0.1322],
        [0.4409],
        [0.2866],
        [0.3800],
        [0.2597],
        [0.1458],
        [0.2297],
        [0.5008],
        [0.2822]], dtype=torch.float64)


[(tensor([0.0566, 0.6353], dtype=torch.float64),
  tensor([0.2994], dtype=torch.float64)),
 (tensor([0.5570, 0.0998], dtype=torch.float64),
  tensor([0.1322], dtype=torch.float64)),
 (tensor([0.7273, 0.8711], dtype=torch.float64),
  tensor([0.4409], dtype=torch.float64)),
 (tensor([0.8842, 0.3436], dtype=torch.float64),
  tensor([0.2866], dtype=torch.float64)),
 (tensor([0.5240, 0.9281], dtype=torch.float64),
  tensor([0.3800], dtype=torch.float64)),
 (tensor([0.9555, 0.1921], dtype=torch.float64),
  tensor([0.2597], dtype=torch.float64)),
 (tensor([0.3800, 0.5099], dtype=torch.float64),
  tensor([0.1458], dtype=torch.float64)),
 (tensor([0.8842, 0.2070], dtype=torch.float64),
  tensor([0.2297], dtype=torch.float64)),
 (tensor([0.7848, 0.9574], dtype=torch.float64),
  tensor([0.5008], dtype=torch.float64)),
 (tensor([0.5976, 0.6198], dtype=torch.float64),
  tensor([0.2822], dtype=torch.float64))]

In [7]:
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf
import torch.nn.functional as F


# thus, we rate the variables accordingly: A value closer to 0.34, 0.2 is rated with a lower number


train_Y = standardize(Y)
#print(train_Y)
#
gp = SingleTaskGP(X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)


UCB = UpperConfidenceBound(gp, beta=0.1)


bounds = torch.stack([torch.zeros(2), torch.ones(2)])
candidate, acq_value = optimize_acqf(
    UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20,
)

print(candidate)



tensor([[1., 1.]])


In [ ]:
def optimize_qehvi(model, train_obj, sampler):
    """Optimizes the qEHVI acquisition function, and returns a new candidate and observation."""
    # partition non-dominated space into disjoint rectangles
    partitioning = NondominatedPartitioning(ref_point=ref_point, Y=train_obj)
    acq_func = qExpectedHypervolumeImprovement(
        model=model,
        ref_point=ref_point.tolist(),  # use known reference point
        partitioning=partitioning,
        sampler=sampler,
    )
    # optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=problem_bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,  # used for intialization heuristic
        options={"batch_limit": 5, "maxiter": 200, "nonnegative": True},
        sequential=True,
    )
    # observe new values
    new_x =  unnormalize(candidates.detach(), bounds=problem_bounds)
    return new_x